<a href="https://colab.research.google.com/github/AAndryanov/Dashboard-HSE/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install dash
#!pip install dash_bootstrap_components

In [4]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import dash_table

## Работа с данными

In [5]:
df = pd.read_excel('dataset.xlsx')

In [6]:
df = df.drop('Unnamed: 0', axis=1)

In [7]:
df.head(3)

,selected_role,campus,education_level,question_category,user_question,user_filters,question_filters,saiga_answer,giga_answer,winner,comment,contexts,refined_question,refined_answer,refined_contexts,response_time,refined_response_time,comment_status
0,Студент,Нижний Новгород,Бакалавриат,Наука,"Для чего нужно подробное изучение науки, прове...","['Нижний Новгород', 'бакалавриат']",['Наука'],"Я понял ваш вопрос. Вы хотите узнать, почему н...","Подробное изучение науки, проведение исследова...",GigaChat,NaN,[{'text': 'Конкурсы игранты НИУВШЭ Конкурсы и ...,"Я приезжаю из другого города учиться, подскажи...",Точные цены на проживание в общежитии и аренду...,[{'text': 'Общежития НИУ ВШЭ Студентам в 2024 ...,3.517504,3.676078,Непустой
1,Студент,Москва,Бакалавриат,Социальные вопросы,Нужно ли подавать какие-то дополнительные доку...,"['Москва', 'бакалавриат']","['Социальные вопросы', 'Безопасность', 'Общежи...",Для продления социальной стипендии необходимо ...,Дополнительные документы для продления социаль...,Saiga,NaN,[{'text': 'Материальная помощь и материальная ...,NaN,NaN,NaN,3.763305,NaN,Пустой
2,Студент,Нижний Новгород,Бакалавриат,Военка,"я 120см, смогу ли я попасть на военку?","['Нижний Новгород', 'бакалавриат']",['ВУЦ'],Я понял ваш страх по поводу возможности поступ...,Информация о требованиях для участия в военной...,Saiga,NaN,[{'text': 'енной службы с зачислением в запас ...,есть ли в вышке супер компьютеры?ещё раз! ответь,"Что-то в вашем вопросе меня смущает. Может, по...",[{'text': 'Суперкомпьютер HARISMa [Искусственн...,1.813993,1.851495,Непустой


## Работа с графиками

In [8]:
coolors = ['#4da2f1', '#ff3d64', '#8ad554', '#ffc636']

In [9]:
def create_campus_plot(df):
  temp = df.campus.value_counts().reset_index()
  fig = px.pie(
      temp,
      names='campus',
      values='count',
      title='Кампусы',
      labels={'campus': 'Кампус', 'count': 'Количество'},
      color_discrete_sequence=coolors,
      hover_name='campus',
      hole=0.65
  )

  fig.update_traces(
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [10]:
fig_campus = create_campus_plot(df)
fig_campus.show(config={'displayModeBar': False})

In [11]:
def create_education_plot(df):
  temp = education_distribution = df.education_level.value_counts().reset_index()

  fig = px.pie(
      temp,
      names='education_level',
      values='count',
      labels={'education_level': 'Уровень образования', 'count': 'Количество'},
      color_discrete_sequence=coolors,
      title='Образование',
      hover_name='education_level',
      hole=0.65
  )

  fig.update_traces(
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [12]:
fig_education = create_education_plot(df)
fig_education.show(config={'displayModeBar': False})

In [13]:
def create_categories_plot(df):
  temp = df.question_category.value_counts().reset_index().sort_values(by='count', ascending=True)

  fig = px.bar(
      temp,
      y='question_category',
      x='count',
      title='Категории вопросов',
      color_discrete_sequence=coolors,
      labels={'question_category': '', 'count': 'Количество'},
      hover_name='question_category'
  )

  fig.update_layout(
      plot_bgcolor='white',
      paper_bgcolor='white',
      title_x=0,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      yaxis=dict(
          showgrid=True,
          gridcolor='lightgray',
      ),
      xaxis=dict(
          showgrid=True,
          gridcolor='lightgray',
      ),
      #height=650,
      #width=1450
  )

  fig.update_traces(
      width=0.7,
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [14]:
fig_categories = create_categories_plot(df)
fig_categories.show(config={'displayModeBar': False})

## Карточки метрик

In [15]:
metric1_card = dbc.Col(
    html.Div(
        [
            html.H6('Процент удовлетворенных пользователей'),
            html.H1('57,18%')
        ],
    ),
    width=4
)

metric2_card = dbc.Col(
    html.Div(
        [
            html.H6('Оценка производительности чат-бота'),
            html.H1('59,63%')
        ],
    ),
    width=4
)

metric3_card = dbc.Col(
    html.Div(
        [
            html.H6('Вероятность галлюцинации'),
            html.H1('15,51%')
        ],
    ),
    width=4
)

## Категории

In [28]:
filters_row = dbc.Row([
    dbc.Col([
        dcc.Dropdown(
            id='campus-filter',
            options=campuses,
            value=['all'],
            multi=True,
            placeholder='Выберите кампус(ы)'
        )
    ], width=4),

    dbc.Col([
        dcc.Dropdown(
            id='education-filter',
            options=education_levels,
            value=['all'],
            multi=True,
            placeholder='Выберите уровень образования'
        )
    ], width=4),

    dbc.Col([
        dcc.Dropdown(
            id='category-filter',
            options=categories,
            value=['all'],
            multi=True,
            placeholder='Выберите категории вопросов'
        )
    ], width=4)
], style={'margin': '20px 0'})

## Дашборд

In [35]:
campuses = [{'label': 'Все', 'value': 'all'}] + [{'label': i, 'value': i} for i in df['campus'].unique()]
education_levels = [{'label': 'Все', 'value': 'all'}] + [{'label': i, 'value': i} for i in df['education_level'].unique()]
categories = [{'label': 'Все', 'value': 'all'}] + [{'label': i, 'value': i} for i in df['question_category'].unique()]

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2('Анализ работы ИИ чат-бота'),

    # Строка с фильтрами
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='campus-filter',
                options=campuses,
                value=['all'],
                multi=True,
                placeholder='Выберите кампус(ы)'
            )
        ], width=4),

        dbc.Col([
            dcc.Dropdown(
                id='education-filter',
                options=education_levels,
                value=['all'],
                multi=True,
                placeholder='Выберите уровень образования'
            )
        ], width=4),

        dbc.Col([
            dcc.Dropdown(
                id='category-filter',
                options=categories,
                value=['all'],
                multi=True,
                placeholder='Выберите категории вопросов'
            )
        ], width=4)
    ], style={'margin': '20px 0'}),

    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='Общая информация', value='tab-1'),
        dcc.Tab(label='Вопросы', value='tab-2'),
        dcc.Tab(label='Работа модели', value='tab-3'),
        dcc.Tab(label='Выгрузка данных', value='tab-4'),
    ]),

    html.Div(id='tabs-content')
])

# Колбэк для фильтрации данных и обновления контента
@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value'),
     Input('campus-filter', 'value'),
     Input('education-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_content(tab, selected_campuses, selected_educations, selected_categories):
    # Фильтрация данных
    filtered_df = df.copy()

    if 'all' not in selected_campuses:
        filtered_df = filtered_df[filtered_df['campus'].isin(selected_campuses)]

    if 'all' not in selected_educations:
        filtered_df = filtered_df[filtered_df['education_level'].isin(selected_educations)]

    if 'all' not in selected_categories:
        filtered_df = filtered_df[filtered_df['question_category'].isin(selected_categories)]

    # Расчет метрик
    total_users = len(filtered_df)
    satisfied_users = len(filtered_df[filtered_df['comment_status'] == 'Пустой'])
    satisfaction_rate = round((satisfied_users / total_users) * 100, 2) if total_users > 0 else 0

    if tab == 'tab-1':
        # Создаем карточки с метриками
        metrics_row = dbc.Row([
            dbc.Col(html.Div([
                html.H6('Процент удовлетворенных пользователей'),
                html.H1(f'{satisfaction_rate}%')
            ]), width=4),

            dbc.Col(html.Div([
                html.H6('Оценка производительности чат-бота'),
                html.H1('59,63%')
            ]), width=4),

            dbc.Col(html.Div([
                html.H6('Вероятность галлюцинации'),
                html.H1('15,51%')
            ]), width=4)
        ])

        # Создаем графики с отфильтрованными данными
        graphs_row1 = dbc.Row([
            dbc.Col(
                dcc.Graph(
                    figure=create_campus_plot(filtered_df),
                    config={'displayModeBar': False},
                    responsive=True,
                ),
                width=6
            ),
            dbc.Col(
                dcc.Graph(
                    figure=create_education_plot(filtered_df),
                    config={'displayModeBar': False},
                    responsive=True,
                ),
                width=6
            )
        ])

        graphs_row2 = dbc.Row([
            dbc.Col(
                dcc.Graph(
                    figure=create_categories_plot(filtered_df),
                    config={'displayModeBar': False},
                    responsive=True,
                ),
                width=11
            )
        ])

        return html.Div([
            metrics_row,
            graphs_row1,
            graphs_row2
        ])

    elif tab == 'tab-2':
        return html.Div("Содержимое вкладки 'Вопросы'")
    elif tab == 'tab-3':
        return html.Div("Содержимое вкладки 'Работа модели'")
    elif tab == 'tab-4':
        return html.Div("Содержимое вкладки 'Выгрузка данных'")

if __name__ == '__main__':
    app.run(debug=False)

<IPython.core.display.Javascript object>

In [22]:
temp = df.question_category.value_counts().reset_index().sort_values(by='count', ascending=True)